In [1]:
!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.9 MB/s eta 0:00:00


In [2]:
# Import relevant packages
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import sys

In [3]:
# We first create a model which containts edges of the graph
model = BayesianModel([('Smoking', 'Yellow Fingers'), ('Smoking', 'Cancer'), ('Solar Flares', 'Radiation'), ('Using Microwave', 'Radiation'), ('Radiation', 'Skin Burn'), ('Radiation', 'Cancer')])

# Enter conditional probability distribution for each variable

# Prior probability for Smoking P(S)
cpd_S = TabularCPD(variable='Smoking', variable_card=2, values=[[0.8], [0.2]])

# Prior probability for Solar Flares P(F)
cpd_F = TabularCPD(variable='Solar Flares', variable_card=2, values=[[0.999], [0.001]])

# Prior probability for Using Microwave P(M)
cpd_M = TabularCPD(variable='Using Microwave', variable_card=2, values=[[0.1], [0.9]])

In [4]:
# Conditional probability for Skin Burn or P(B|R)
cpd_B = TabularCPD(variable='Skin Burn',variable_card=2, values = [[0.99, 0.9], [0.01, 0.1]],
                                  evidence = ['Radiation'],
                                  evidence_card=[2])

# Conditional probability for Radiation or P(R|F, M)
cpd_R = TabularCPD(variable='Radiation', variable_card=2, values =  [[0.99, 0.9, 0.7, 0.5], [0.01, 0.1, 0.3, 0.5]],
                                  evidence = ['Solar Flares', 'Using Microwave'],
                                  evidence_card=[2, 2])
# Conditional probability for Cancer or P(C|S, R)
cpd_C = TabularCPD(variable='Cancer', variable_card=2, values = [[0.9, 0.4, 0.8, 0.1], [0.1, 0.6, 0.2, 0.9]],
                                  evidence = ['Smoking', 'Radiation'],
                                  evidence_card=[2, 2])

# Conditional probability for Yellow Fingers or P(Y|S)
cpd_Y = TabularCPD(variable='Yellow Fingers',variable_card=2, values = [[0.9, 0.1], [0.1, 0.9]],
                                  evidence = ['Smoking'],
                                  evidence_card=[2])

model.add_cpds(cpd_B, cpd_R, cpd_C, cpd_S, cpd_F, cpd_M, cpd_Y)

In [5]:
print(model.check_model())

True


In [6]:
from pgmpy.inference import VariableElimination
# Going to do variable elimination
infer = VariableElimination(model)

# Compute probability of Radiation given Cancer = 1
phi_query = infer.query(['Radiation'], evidence={'Cancer':1}, joint = False)
factor = phi_query['Radiation']
print('Probability of Radiation given Cancer = 1')
print(factor)

# Compute probability of Cancer given Skin Burn = 1
phi_query = infer.query(['Cancer'], evidence={'Skin Burn':1}, joint = False)
factor = phi_query['Cancer']
print('Probability of Cancer given Skin Burn = 1')
print(factor)

# Compute probability of Cancer given Using Microwave = 0
phi_query = infer.query(['Cancer'], evidence={'Using Microwave':0}, joint = False)
factor = phi_query['Cancer']
print('Probability of Cancer given Using Microwave = 0')
print(factor)

Probability of Radiation given Cancer = 1
+--------------+------------------+
| Radiation    |   phi(Radiation) |
+==============+==================+
| Radiation(0) |           0.6438 |
+--------------+------------------+
| Radiation(1) |           0.3562 |
+--------------+------------------+
Probability of Cancer given Skin Burn = 1
+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.6092 |
+-----------+---------------+
| Cancer(1) |        0.3908 |
+-----------+---------------+
Probability of Cancer given Using Microwave = 0
+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.8744 |
+-----------+---------------+
| Cancer(1) |        0.1256 |
+-----------+---------------+
